In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from datetime import date, datetime

from sklearn.metrics import confusion_matrix
from sklearn.metrics import normalized_mutual_info_score, mutual_info_score
from sklearn.feature_selection import mutual_info_regression

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline

from matplotlib import colors
import seaborn as sns


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [3]:
str_state = 'GA'
file = 'agent_comments_sales_'+str_state
#file = 'agent_comments_testdf'
#READ THE SALES DATA - STATE



In [4]:
df_sales = pd.read_feather(file + '_sentencecomposition.fea')
#df_sales = pd.read_feather(file + '_withsentiment.fea')
#df_sales = pd.read_feather(file+'_processed.fea')


In [9]:
df_sales['Listing_error'] = (df_sales['Transprice']-df_sales['ListingPrice'])\
                                                    /df_sales['Transprice']

In [16]:
df_sales[(df_sales['Listing_error']<-9) & (df_sales['Listing_error']>-9.5)][['index', 'listingid', 'asgpropid', 'qtr', 'tax_year', 'cbsa_div',
       'state', 'Transdate', 'Transprice', 'bedrooms', 'total_value', 
       'ListingPrice', 'avmValue', 'avmerror', 'rentsale']]

,index,listingid,asgpropid,qtr,tax_year,cbsa_div,state,Transdate,Transprice,bedrooms,total_value,ListingPrice,avmValue,avmerror,rentsale
16906,922331,6112254,27704746.0,201805,2017.0,12060,GA,20180618,251000.0,5.0,223400.0,2545817.0,264742.02,-0.054749,S
18911,924339,139470602,27452850.0,201802,2017.0,12060,GA,20180216,67000.0,4.0,33300.0,689000.0,40154.36,0.400681,S
27484,933024,139453438,27127296.0,201802,2017.0,12060,GA,20180315,149000.0,3.0,106000.0,1495500.0,137042.31,0.080253,S
42389,948136,127137767,27018797.0,201802,2017.0,12060,GA,20180126,182000.0,5.0,157400.0,1900000.0,196748.47,-0.081036,S
44779,950576,139775648,25957869.0,201802,2017.0,12060,GA,20180314,170000.0,3.0,117640.0,1729000.0,146934.38,0.135680,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591808,1504836,77853117,26886434.0,200702,2013.0,12060,GA,20070131,165000.0,4.0,63780.0,1689000.0,164365.76,0.003844,S
595504,1510617,77872588,27207385.0,200705,2013.0,12060,GA,20070515,97000.0,2.0,65438.0,999000.0,109540.41,-0.129283,S
604113,2299802,4886345,28022074.0,201808,2017.0,12020,GA,20180725,237000.0,NaN,241893.0,2450000.0,293908.10,-0.240119,S
617924,3792021,85530454,28079899.0,201608,2015.0,29300,GA,20160808,115000.0,3.0,105216.0,1199000.0,117967.67,-0.025806,S


In [5]:
df_sales_embedding = pd.read_feather(file + '_withembeddings.fea')

In [6]:
df_sales.columns

Index(['index', 'listingid', 'asgpropid', 'qtr', 'tax_year', 'cbsa_div',
       'cbsa', 'state', 'Transdate', 'Transprice', 'bedrooms', 'total_value',
       'cj_living_area', 'basement', 'has_pool', 'parking', 'is_poor',
       'is_good', 'HAS_VIEW', 'has_golf', 'has_water', 'has_woods', 'has_hill',
       'fips_code', 'census_tract', 'zip', 'effective_year_built', 'story',
       'ListingPrice', 'publicremarks', 'avmValue', 'avmerror', 'rentsale',
       'publicremarks_prepared', 'publicremarks_prepared_spellcheck',
       'publicremarks_prepared_unicode', 'clean_publicremarks',
       'clean_publicremarks_additional_words', 'clean_publicremarks_nopunct',
       'clean_publicremarks_nopunct_additional_words',
       'clean_publicremarks_stemmed',
       'clean_publicremarks_stemmed_additional_words',
       'clean_publicremarks_lemmatized',
       'clean_publicremarks_lemmatized_additional_words',
       'publicremarks_nltkpolarity', 'clean_publicremarks_nltkpolarity',
       'clean_

In [7]:
df_sales_embedding.columns

Index(['index', 'listingid', 'asgpropid', 'qtr', 'tax_year', 'cbsa_div',
       'cbsa', 'state', 'Transdate', 'Transprice', 'bedrooms', 'total_value',
       'cj_living_area', 'basement', 'has_pool', 'parking', 'is_poor',
       'is_good', 'HAS_VIEW', 'has_golf', 'has_water', 'has_woods', 'has_hill',
       'fips_code', 'census_tract', 'zip', 'effective_year_built', 'story',
       'ListingPrice', 'publicremarks', 'avmValue', 'avmerror', 'rentsale',
       'publicremarks_prepared', 'publicremarks_prepared_unicode',
       'clean_publicremarks', 'clean_publicremarks_additional_words',
       'clean_publicremarks_nopunct', 'clean_publicremarks_stemmed',
       'clean_publicremarks_lemmatized', 'publicremarks_nltkpolarity',
       'clean_publicremarks_nltkpolarity',
       'clean_publicremarks_additional_words_nltkpolarity',
       'clean_publicremarks_nopunct_nltkpolarity',
       'clean_publicremarks_stemmed_nltkpolarity',
       'clean_publicremarks_lemmatized_nltkpolarity',
       'pu

In [6]:
df_sales_embedding.head(5)

,index,listingid,asgpropid,qtr,tax_year,cbsa_div,cbsa,state,Transdate,Transprice,...,clean_publicremarks_additional_words_vaderpolarity,clean_publicremarks_nopunct_vaderpolarity,clean_publicremarks_stemmed_vaderpolarity,clean_publicremarks_lemmatized_vaderpolarity,publicremarks_embeddings,clean_publicremarks_embeddings,clean_publicremarks_additional_words_embeddings,clean_publicremarks_nopunct_embeddings,clean_publicremarks_stemmed_embeddings,clean_publicremarks_lemmatized_embeddings
0,128,226661582,27649366.0,202008,2020.0,23580,23580,GA,20200811,359000.0,...,"{'compound': 0.9401, 'neg': 0.0, 'neu': 0.803,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.6486, 'neg': 0.0, 'neu': 0.936,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","[-0.018459855, -0.023812085, -0.043892566, -0....","[-0.052161112, -0.0555884, -0.057574343, -0.02...","[-0.050703425, -0.057770006, -0.053797793, -0....","[-0.052161112, -0.0555884, -0.057574343, -0.02...","[-0.03600466, -0.059054498, -0.06483389, -0.02...","[-0.050262094, -0.05206799, -0.05940456, -0.02..."
1,190,226786904,27628370.0,202008,2020.0,29300,29300,GA,20200925,265000.0,...,"{'compound': 0.9876, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9889, 'neg': 0.014, 'neu': 0.69...","{'compound': 0.9732, 'neg': 0.0, 'neu': 0.801,...","{'compound': 0.9875, 'neg': 0.014, 'neu': 0.70...","[-0.060909368, -0.044562507, -0.048587322, -0....","[-0.06049352, -0.05438737, -0.048537098, -0.05...","[-0.059828583, -0.05020684, -0.046291806, -0.0...","[-0.06049352, -0.05438737, -0.048537098, -0.05...","[-0.065291695, -0.06122866, -0.04930517, 0.006...","[-0.060174935, -0.05459462, -0.045611445, -0.0..."
2,232,226142320,27963359.0,202005,2020.0,27600,27600,GA,20200624,239900.0,...,"{'compound': 0.8221, 'neg': 0.0, 'neu': 0.851,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.958,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","[0.051360507, -0.007422204, -0.054501485, -0.0...","[0.00015653955, -0.017207557, -0.06090085, -0....","[0.0069589866, -0.016838172, -0.06001686, -0.0...","[0.00015653955, -0.017207557, -0.06090085, -0....","[0.003589608, -0.049508873, -0.06795308, -0.06...","[-0.004722175, -0.03370639, -0.06371849, -0.06..."
3,308,226738459,27516267.0,202008,2020.0,29300,29300,GA,20200713,130900.0,...,"{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","[-0.020899976, 0.016856506, -0.011256389, -0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0...","[0.0036934454, 0.047616564, -0.0056924997, 0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0...","[-0.050069433, 0.007041185, -0.033259477, -0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0..."
4,467,226850183,27791872.0,202008,2020.0,23580,23580,GA,20200702,406000.0,...,"{'compound': 0.8309, 'neg': 0.0, 'neu': 0.878,...","{'compound': 0.8658, 'neg': 0.0, 'neu': 0.857,...","{'compound': 0.5106, 'neg': 0.0, 'neu': 0.932,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.876, ...","[-0.047313046, 0.00859506, -0.04837404, -0.052...","[-0.055047132, -0.031499583, -0.055006824, -0....","[-0.053909175, -0.025725972, -0.05363282, -0.0...","[-0.055047132, -0.031499583, -0.055006824, -0....","[-0.061100535, -0.05123385, -0.067388244, -0.0...","[-0.054386344, -0.04275706, -0.052802, -0.0314..."


In [7]:
df_sales.head(5)

,index,listingid,asgpropid,qtr,tax_year,cbsa_div,cbsa,state,Transdate,Transprice,...,publicremarks_vaderpolarity,clean_publicremarks_vaderpolarity,clean_publicremarks_additional_words_vaderpolarity,clean_publicremarks_nopunct_vaderpolarity,clean_publicremarks_nopunct_additional_words_vaderpolarity,clean_publicremarks_stemmed_vaderpolarity,clean_publicremarks_stemmed_additional_words_vaderpolarity,clean_publicremarks_lemmatized_vaderpolarity,clean_publicremarks_lemmatized_additional_words_vaderpolarity,sentence_composition
0,128,226661582,27649366.0,202008,2020.0,23580,23580,GA,20200811,359000.0,...,"{'compound': 0.9627, 'neg': 0.0, 'neu': 0.86, ...","{'compound': 0.9627, 'neg': 0.0, 'neu': 0.781,...","{'compound': 0.9401, 'neg': 0.0, 'neu': 0.803,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","{'compound': 0.6486, 'neg': 0.0, 'neu': 0.936,...","{'compound': 0.4939, 'neg': 0.0, 'neu': 0.954,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","[55.00000000000001, 13.750000000000002, 17.5, ..."
1,190,226786904,27628370.0,202008,2020.0,29300,29300,GA,20200925,265000.0,...,"{'compound': 0.9939, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9898, 'neg': 0.013, 'neu': 0.70...","{'compound': 0.9876, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9889, 'neg': 0.014, 'neu': 0.69...","{'compound': 0.9864, 'neg': 0.016, 'neu': 0.7,...","{'compound': 0.9732, 'neg': 0.0, 'neu': 0.801,...","{'compound': 0.9686, 'neg': 0.0, 'neu': 0.799,...","{'compound': 0.9875, 'neg': 0.014, 'neu': 0.70...","{'compound': 0.9844, 'neg': 0.016, 'neu': 0.70...","[60.18518518518518, 13.88888888888889, 18.5185..."
2,232,226142320,27963359.0,202005,2020.0,27600,27600,GA,20200624,239900.0,...,"{'compound': 0.8528, 'neg': 0.0, 'neu': 0.893,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.853,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.851,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.958,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.957,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","[58.490566037735846, 9.433962264150944, 26.415..."
3,308,226738459,27516267.0,202008,2020.0,29300,29300,GA,20200713,130900.0,...,"{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","[53.84615384615385, 7.6923076923076925, 38.461..."
4,467,226850183,27791872.0,202008,2020.0,23580,23580,GA,20200702,406000.0,...,"{'compound': 0.8829, 'neg': 0.0, 'neu': 0.877,...","{'compound': 0.8829, 'neg': 0.0, 'neu': 0.849,...","{'compound': 0.8309, 'neg': 0.0, 'neu': 0.878,...","{'compound': 0.8658, 'neg': 0.0, 'neu': 0.857,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.886, ...","{'compound': 0.5106, 'neg': 0.0, 'neu': 0.932,...","{'compound': 0.2023, 'neg': 0.0, 'neu': 0.969,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.876, ...","{'compound': 0.6908, 'neg': 0.0, 'neu': 0.908,...","[65.57377049180327, 13.114754098360656, 11.475..."


In [8]:
df_sales_embedding_use = df_sales_embedding[['index','publicremarks_embeddings', 'clean_publicremarks_embeddings',
                                               'clean_publicremarks_additional_words_embeddings',
                                               'clean_publicremarks_nopunct_embeddings',
                                               'clean_publicremarks_stemmed_embeddings',
                                               'clean_publicremarks_lemmatized_embeddings']]

In [9]:
df_sales_embedding_use.head(5)

,index,publicremarks_embeddings,clean_publicremarks_embeddings,clean_publicremarks_additional_words_embeddings,clean_publicremarks_nopunct_embeddings,clean_publicremarks_stemmed_embeddings,clean_publicremarks_lemmatized_embeddings
0,128,"[-0.018459855, -0.023812085, -0.043892566, -0....","[-0.052161112, -0.0555884, -0.057574343, -0.02...","[-0.050703425, -0.057770006, -0.053797793, -0....","[-0.052161112, -0.0555884, -0.057574343, -0.02...","[-0.03600466, -0.059054498, -0.06483389, -0.02...","[-0.050262094, -0.05206799, -0.05940456, -0.02..."
1,190,"[-0.060909368, -0.044562507, -0.048587322, -0....","[-0.06049352, -0.05438737, -0.048537098, -0.05...","[-0.059828583, -0.05020684, -0.046291806, -0.0...","[-0.06049352, -0.05438737, -0.048537098, -0.05...","[-0.065291695, -0.06122866, -0.04930517, 0.006...","[-0.060174935, -0.05459462, -0.045611445, -0.0..."
2,232,"[0.051360507, -0.007422204, -0.054501485, -0.0...","[0.00015653955, -0.017207557, -0.06090085, -0....","[0.0069589866, -0.016838172, -0.06001686, -0.0...","[0.00015653955, -0.017207557, -0.06090085, -0....","[0.003589608, -0.049508873, -0.06795308, -0.06...","[-0.004722175, -0.03370639, -0.06371849, -0.06..."
3,308,"[-0.020899976, 0.016856506, -0.011256389, -0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0...","[0.0036934454, 0.047616564, -0.0056924997, 0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0...","[-0.050069433, 0.007041185, -0.033259477, -0.0...","[-0.028272344, -0.005281781, -0.0073222406, -0..."
4,467,"[-0.047313046, 0.00859506, -0.04837404, -0.052...","[-0.055047132, -0.031499583, -0.055006824, -0....","[-0.053909175, -0.025725972, -0.05363282, -0.0...","[-0.055047132, -0.031499583, -0.055006824, -0....","[-0.061100535, -0.05123385, -0.067388244, -0.0...","[-0.054386344, -0.04275706, -0.052802, -0.0314..."


In [10]:
df_sales_use = df_sales[['index','Transprice', 'bedrooms', 'cj_living_area', 'basement', 'has_pool', 'parking', 
       'story',  'avmValue', 'avmerror', 
       'publicremarks_nltkpolarity', 'clean_publicremarks_nltkpolarity',
       'clean_publicremarks_additional_words_nltkpolarity',
       'clean_publicremarks_nopunct_nltkpolarity',
       'clean_publicremarks_nopunct_additional_words_nltkpolarity',
       'clean_publicremarks_stemmed_nltkpolarity',
       'clean_publicremarks_stemmed_additional_words_nltkpolarity',
       'clean_publicremarks_lemmatized_nltkpolarity',
       'clean_publicremarks_lemmatized_additional_words_nltkpolarity',
       'publicremarks_textblobpolarity',
       'clean_publicremarks_textblobpolarity',
       'clean_publicremarks_additional_words_textblobpolarity',
       'clean_publicremarks_nopunct_textblobpolarity',
       'clean_publicremarks_nopunct_additional_words_textblobpolarity',
       'clean_publicremarks_stemmed_textblobpolarity',
       'clean_publicremarks_stemmed_additional_words_textblobpolarity',
       'clean_publicremarks_lemmatized_textblobpolarity',
       'clean_publicremarks_lemmatized_additional_words_textblobpolarity',
       'publicremarks_vaderpolarity', 'clean_publicremarks_vaderpolarity',
       'clean_publicremarks_additional_words_vaderpolarity',
       'clean_publicremarks_nopunct_vaderpolarity',
       'clean_publicremarks_nopunct_additional_words_vaderpolarity',
       'clean_publicremarks_stemmed_vaderpolarity',
       'clean_publicremarks_stemmed_additional_words_vaderpolarity',
       'clean_publicremarks_lemmatized_vaderpolarity',
       'clean_publicremarks_lemmatized_additional_words_vaderpolarity',
       'sentence_composition']]

In [11]:
df_sales_use.columns

Index(['index', 'Transprice', 'bedrooms', 'cj_living_area', 'basement',
       'has_pool', 'parking', 'story', 'avmValue', 'avmerror',
       'publicremarks_nltkpolarity', 'clean_publicremarks_nltkpolarity',
       'clean_publicremarks_additional_words_nltkpolarity',
       'clean_publicremarks_nopunct_nltkpolarity',
       'clean_publicremarks_nopunct_additional_words_nltkpolarity',
       'clean_publicremarks_stemmed_nltkpolarity',
       'clean_publicremarks_stemmed_additional_words_nltkpolarity',
       'clean_publicremarks_lemmatized_nltkpolarity',
       'clean_publicremarks_lemmatized_additional_words_nltkpolarity',
       'publicremarks_textblobpolarity',
       'clean_publicremarks_textblobpolarity',
       'clean_publicremarks_additional_words_textblobpolarity',
       'clean_publicremarks_nopunct_textblobpolarity',
       'clean_publicremarks_nopunct_additional_words_textblobpolarity',
       'clean_publicremarks_stemmed_textblobpolarity',
       'clean_publicremarks_stemmed

In [12]:
columns_to_list = ['publicremarks_nltkpolarity', 
                   'clean_publicremarks_nltkpolarity',
                   'clean_publicremarks_additional_words_nltkpolarity',
                   'clean_publicremarks_nopunct_nltkpolarity',
                   'clean_publicremarks_nopunct_additional_words_nltkpolarity',
                   'clean_publicremarks_stemmed_nltkpolarity',
                   'clean_publicremarks_stemmed_additional_words_nltkpolarity',
                   'clean_publicremarks_lemmatized_nltkpolarity',
                   'clean_publicremarks_lemmatized_additional_words_nltkpolarity']

In [13]:
for column in columns_to_list:
    df_sales_use[column] =df_sales_use[column].apply(lambda x: list(x.values()))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_sales_use

,index,Transprice,bedrooms,cj_living_area,basement,has_pool,parking,story,avmValue,avmerror,...,publicremarks_vaderpolarity,clean_publicremarks_vaderpolarity,clean_publicremarks_additional_words_vaderpolarity,clean_publicremarks_nopunct_vaderpolarity,clean_publicremarks_nopunct_additional_words_vaderpolarity,clean_publicremarks_stemmed_vaderpolarity,clean_publicremarks_stemmed_additional_words_vaderpolarity,clean_publicremarks_lemmatized_vaderpolarity,clean_publicremarks_lemmatized_additional_words_vaderpolarity,sentence_composition
0,128,359000.0,4.0,3295.0,1,0,2,1.5,405972.57,-0.130843,...,"{'compound': 0.9627, 'neg': 0.0, 'neu': 0.86, ...","{'compound': 0.9627, 'neg': 0.0, 'neu': 0.781,...","{'compound': 0.9401, 'neg': 0.0, 'neu': 0.803,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","{'compound': 0.6486, 'neg': 0.0, 'neu': 0.936,...","{'compound': 0.4939, 'neg': 0.0, 'neu': 0.954,...","{'compound': 0.9578, 'neg': 0.0, 'neu': 0.786,...","{'compound': 0.93, 'neg': 0.0, 'neu': 0.81, 'p...","[55.00000000000001, 13.750000000000002, 17.5, ..."
1,190,265000.0,4.0,2927.0,2,0,0,1.0,244827.67,0.076122,...,"{'compound': 0.9939, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9898, 'neg': 0.013, 'neu': 0.70...","{'compound': 0.9876, 'neg': 0.015, 'neu': 0.70...","{'compound': 0.9889, 'neg': 0.014, 'neu': 0.69...","{'compound': 0.9864, 'neg': 0.016, 'neu': 0.7,...","{'compound': 0.9732, 'neg': 0.0, 'neu': 0.801,...","{'compound': 0.9686, 'neg': 0.0, 'neu': 0.799,...","{'compound': 0.9875, 'neg': 0.014, 'neu': 0.70...","{'compound': 0.9844, 'neg': 0.016, 'neu': 0.70...","[60.18518518518518, 13.88888888888889, 18.5185..."
2,232,239900.0,4.0,2230.0,0,0,2,1.0,229674.21,0.042625,...,"{'compound': 0.8528, 'neg': 0.0, 'neu': 0.893,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.853,...","{'compound': 0.8221, 'neg': 0.0, 'neu': 0.851,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.958,...","{'compound': 0.3182, 'neg': 0.0, 'neu': 0.957,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.858,...","{'compound': 0.8074, 'neg': 0.0, 'neu': 0.855,...","[58.490566037735846, 9.433962264150944, 26.415..."
3,308,130900.0,3.0,1333.0,0,0,2,1.0,143607.40,-0.097077,...,"{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos...","[53.84615384615385, 7.6923076923076925, 38.461..."
4,467,406000.0,4.0,3246.0,0,0,2,2.0,408971.22,-0.007318,...,"{'compound': 0.8829, 'neg': 0.0, 'neu': 0.877,...","{'compound': 0.8829, 'neg': 0.0, 'neu': 0.849,...","{'compound': 0.8309, 'neg': 0.0, 'neu': 0.878,...","{'compound': 0.8658, 'neg': 0.0, 'neu': 0.857,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.886, ...","{'compound': 0.5106, 'neg': 0.0, 'neu': 0.932,...","{'compound': 0.2023, 'neg': 0.0, 'neu': 0.969,...","{'compound': 0.802, 'neg': 0.0, 'neu': 0.876, ...","{'compound': 0.6908, 'neg': 0.0, 'neu': 0.908,...","[65.57377049180327, 13.114754098360656, 11.475..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636144,6908610,163900.0,NaN,1616.0,0,0,2,1.0,174640.18,-0.065529,...,"{'compound': 0.7684, 'neg': 0.0, 'neu': 0.832,...","{'compound': 0.6351, 'neg': 0.0, 'neu': 0.834,...","{'compound': 0.6351, 'neg': 0.0, 'neu': 0.822,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.85, ...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0.839,...","{'compound': 0.5574, 'neg': 0.0, 'neu': 0

In [15]:
#list(df_sales_use['publicremarks_nltkpolarity'][0].values())

In [16]:
columns_to_list = ['publicremarks_vaderpolarity', 'clean_publicremarks_vaderpolarity',
                   'clean_publicremarks_additional_words_vaderpolarity',
                   'clean_publicremarks_nopunct_vaderpolarity',
                   'clean_publicremarks_nopunct_additional_words_vaderpolarity',
                   'clean_publicremarks_stemmed_vaderpolarity',
                   'clean_publicremarks_stemmed_additional_words_vaderpolarity',
                   'clean_publicremarks_lemmatized_vaderpolarity',
                   'clean_publicremarks_lemmatized_additional_words_vaderpolarity']

In [17]:
for column in columns_to_list:
    df_sales_use[column] =df_sales_use[column].apply(lambda x: list(x.values()))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
list(df_sales_use.columns)

['index',
 'Transprice',
 'bedrooms',
 'cj_living_area',
 'basement',
 'has_pool',
 'parking',
 'story',
 'avmValue',
 'avmerror',
 'publicremarks_nltkpolarity',
 'clean_publicremarks_nltkpolarity',
 'clean_publicremarks_additional_words_nltkpolarity',
 'clean_publicremarks_nopunct_nltkpolarity',
 'clean_publicremarks_nopunct_additional_words_nltkpolarity',
 'clean_publicremarks_stemmed_nltkpolarity',
 'clean_publicremarks_stemmed_additional_words_nltkpolarity',
 'clean_publicremarks_lemmatized_nltkpolarity',
 'clean_publicremarks_lemmatized_additional_words_nltkpolarity',
 'publicremarks_textblobpolarity',
 'clean_publicremarks_textblobpolarity',
 'clean_publicremarks_additional_words_textblobpolarity',
 'clean_publicremarks_nopunct_textblobpolarity',
 'clean_publicremarks_nopunct_additional_words_textblobpolarity',
 'clean_publicremarks_stemmed_textblobpolarity',
 'clean_publicremarks_stemmed_additional_words_textblobpolarity',
 'clean_publicremarks_lemmatized_textblobpolarity',
 'cl

In [19]:
df_sales_use = df_sales_use.merge(df_sales_embedding_use, on='index')

In [20]:
df_sales_use.columns

Index(['index', 'Transprice', 'bedrooms', 'cj_living_area', 'basement',
       'has_pool', 'parking', 'story', 'avmValue', 'avmerror',
       'publicremarks_nltkpolarity', 'clean_publicremarks_nltkpolarity',
       'clean_publicremarks_additional_words_nltkpolarity',
       'clean_publicremarks_nopunct_nltkpolarity',
       'clean_publicremarks_nopunct_additional_words_nltkpolarity',
       'clean_publicremarks_stemmed_nltkpolarity',
       'clean_publicremarks_stemmed_additional_words_nltkpolarity',
       'clean_publicremarks_lemmatized_nltkpolarity',
       'clean_publicremarks_lemmatized_additional_words_nltkpolarity',
       'publicremarks_textblobpolarity',
       'clean_publicremarks_textblobpolarity',
       'clean_publicremarks_additional_words_textblobpolarity',
       'clean_publicremarks_nopunct_textblobpolarity',
       'clean_publicremarks_nopunct_additional_words_textblobpolarity',
       'clean_publicremarks_stemmed_textblobpolarity',
       'clean_publicremarks_stemmed

In [21]:
df_sales_use['clean_publicremarks_nopunct_textblobpolarity']

0         [0.24625850340136055, 0.45442176870748296]
1          [0.22595708639186904, 0.4931109228935316]
2         [0.10000000000000002, 0.35833333333333334]
3         [0.13636363636363635, 0.45454545454545453]
4         [0.18444444444444447, 0.32666666666666666]
                             ...                    
636144      [0.2965909090909091, 0.6011363636363637]
636145      [0.3906060606060606, 0.5825757575757576]
636146     [0.35909090909090907, 0.5511363636363636]
636147                                   [0.2, 0.65]
636148      [0.3651515151515152, 0.6282828282828282]
Name: clean_publicremarks_nopunct_textblobpolarity, Length: 636149, dtype: object

In [22]:
for column in list(df_sales_use.columns):
    print(df_sales_use[column][0])

128
359000.0
4.0
3295.0
1
0
2
1.5
405972.57
-0.13084281337047354
[0.9627, 0.0, 0.856, 0.144]
[0.9627, 0.0, 0.778, 0.222]
[0.9401, 0.0, 0.801, 0.199]
[0.9578, 0.0, 0.786, 0.214]
[0.93, 0.0, 0.81, 0.19]
[0.6486, 0.0, 0.936, 0.064]
[0.4939, 0.0, 0.954, 0.046]
[0.9578, 0.0, 0.786, 0.214]
[0.93, 0.0, 0.81, 0.19]
[0.21617063 0.4156746 ]
[0.25816327 0.45442177]
[0.2366369  0.49619048]
[0.2462585  0.45442177]
[0.2147619  0.49619048]
[0.04204545 0.2375    ]
[0.05606061 0.31666667]
[0.2462585  0.45442177]
[0.2147619  0.49619048]
[0.9627, 0.0, 0.86, 0.14]
[0.9627, 0.0, 0.781, 0.219]
[0.9401, 0.0, 0.803, 0.197]
[0.9578, 0.0, 0.786, 0.214]
[0.93, 0.0, 0.81, 0.19]
[0.6486, 0.0, 0.936, 0.064]
[0.4939, 0.0, 0.954, 0.046]
[0.9578, 0.0, 0.786, 0.214]
[0.93, 0.0, 0.81, 0.19]
[55.   13.75 17.5   6.25  2.5   0.    1.25 80.   17.  ]
[-1.84598546e-02 -2.38120854e-02 -4.38925661e-02 -3.90198678e-02
  2.05408223e-03 -6.81159610e-04  1.79503132e-02 -6.32521287e-02
  6.58034310e-02 -6.05727918e-02 -5.70762865e-0

In [23]:
columns_to_split = ['publicremarks_nltkpolarity',
                     'clean_publicremarks_nltkpolarity',
                     'clean_publicremarks_additional_words_nltkpolarity',
                     'clean_publicremarks_nopunct_nltkpolarity',
                     'clean_publicremarks_nopunct_additional_words_nltkpolarity',
                     'clean_publicremarks_stemmed_nltkpolarity',
                     'clean_publicremarks_stemmed_additional_words_nltkpolarity',
                     'clean_publicremarks_lemmatized_nltkpolarity',
                     'clean_publicremarks_lemmatized_additional_words_nltkpolarity',
                     'publicremarks_textblobpolarity',
                     'clean_publicremarks_textblobpolarity',
                     'clean_publicremarks_additional_words_textblobpolarity',
                     'clean_publicremarks_nopunct_textblobpolarity',
                     'clean_publicremarks_nopunct_additional_words_textblobpolarity',
                     'clean_publicremarks_stemmed_textblobpolarity',
                     'clean_publicremarks_stemmed_additional_words_textblobpolarity',
                     'clean_publicremarks_lemmatized_textblobpolarity',
                     'clean_publicremarks_lemmatized_additional_words_textblobpolarity',
                     'publicremarks_vaderpolarity',
                     'clean_publicremarks_vaderpolarity',
                     'clean_publicremarks_additional_words_vaderpolarity',
                     'clean_publicremarks_nopunct_vaderpolarity',
                     'clean_publicremarks_nopunct_additional_words_vaderpolarity',
                     'clean_publicremarks_stemmed_vaderpolarity',
                     'clean_publicremarks_stemmed_additional_words_vaderpolarity',
                     'clean_publicremarks_lemmatized_vaderpolarity',
                     'clean_publicremarks_lemmatized_additional_words_vaderpolarity',
                     'sentence_composition',
                     'publicremarks_embeddings',
                     'clean_publicremarks_embeddings',
                     'clean_publicremarks_additional_words_embeddings',
                     'clean_publicremarks_nopunct_embeddings',
                     'clean_publicremarks_stemmed_embeddings',
                     'clean_publicremarks_lemmatized_embeddings']

In [24]:
import seaborn as sns

In [25]:
df_sales_use2 = df_sales_use

for column in columns_to_split:
    nelements = (len(df_sales_use[column][0]))
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    print(column_names)
    split_df = pd.DataFrame(df_sales_use2[column].tolist(), columns=column_names)
    df_sales_use2 = pd.concat([df_sales_use2, split_df], axis=1)
    df_sales_use2 = df_sales_use2.drop(columns = [column])
    

['publicremarks_nltkpolarity_0', 'publicremarks_nltkpolarity_1', 'publicremarks_nltkpolarity_2', 'publicremarks_nltkpolarity_3']
['clean_publicremarks_nltkpolarity_0', 'clean_publicremarks_nltkpolarity_1', 'clean_publicremarks_nltkpolarity_2', 'clean_publicremarks_nltkpolarity_3']
['clean_publicremarks_additional_words_nltkpolarity_0', 'clean_publicremarks_additional_words_nltkpolarity_1', 'clean_publicremarks_additional_words_nltkpolarity_2', 'clean_publicremarks_additional_words_nltkpolarity_3']
['clean_publicremarks_nopunct_nltkpolarity_0', 'clean_publicremarks_nopunct_nltkpolarity_1', 'clean_publicremarks_nopunct_nltkpolarity_2', 'clean_publicremarks_nopunct_nltkpolarity_3']
['clean_publicremarks_nopunct_additional_words_nltkpolarity_0', 'clean_publicremarks_nopunct_additional_words_nltkpolarity_1', 'clean_publicremarks_nopunct_additional_words_nltkpolarity_2', 'clean_publicremarks_nopunct_additional_words_nltkpolarity_3']
['clean_publicremarks_stemmed_nltkpolarity_0', 'clean_publi

['clean_publicremarks_embeddings_0', 'clean_publicremarks_embeddings_1', 'clean_publicremarks_embeddings_2', 'clean_publicremarks_embeddings_3', 'clean_publicremarks_embeddings_4', 'clean_publicremarks_embeddings_5', 'clean_publicremarks_embeddings_6', 'clean_publicremarks_embeddings_7', 'clean_publicremarks_embeddings_8', 'clean_publicremarks_embeddings_9', 'clean_publicremarks_embeddings_10', 'clean_publicremarks_embeddings_11', 'clean_publicremarks_embeddings_12', 'clean_publicremarks_embeddings_13', 'clean_publicremarks_embeddings_14', 'clean_publicremarks_embeddings_15', 'clean_publicremarks_embeddings_16', 'clean_publicremarks_embeddings_17', 'clean_publicremarks_embeddings_18', 'clean_publicremarks_embeddings_19', 'clean_publicremarks_embeddings_20', 'clean_publicremarks_embeddings_21', 'clean_publicremarks_embeddings_22', 'clean_publicremarks_embeddings_23', 'clean_publicremarks_embeddings_24', 'clean_publicremarks_embeddings_25', 'clean_publicremarks_embeddings_26', 'clean_pub

['clean_publicremarks_additional_words_embeddings_0', 'clean_publicremarks_additional_words_embeddings_1', 'clean_publicremarks_additional_words_embeddings_2', 'clean_publicremarks_additional_words_embeddings_3', 'clean_publicremarks_additional_words_embeddings_4', 'clean_publicremarks_additional_words_embeddings_5', 'clean_publicremarks_additional_words_embeddings_6', 'clean_publicremarks_additional_words_embeddings_7', 'clean_publicremarks_additional_words_embeddings_8', 'clean_publicremarks_additional_words_embeddings_9', 'clean_publicremarks_additional_words_embeddings_10', 'clean_publicremarks_additional_words_embeddings_11', 'clean_publicremarks_additional_words_embeddings_12', 'clean_publicremarks_additional_words_embeddings_13', 'clean_publicremarks_additional_words_embeddings_14', 'clean_publicremarks_additional_words_embeddings_15', 'clean_publicremarks_additional_words_embeddings_16', 'clean_publicremarks_additional_words_embeddings_17', 'clean_publicremarks_additional_words

['clean_publicremarks_nopunct_embeddings_0', 'clean_publicremarks_nopunct_embeddings_1', 'clean_publicremarks_nopunct_embeddings_2', 'clean_publicremarks_nopunct_embeddings_3', 'clean_publicremarks_nopunct_embeddings_4', 'clean_publicremarks_nopunct_embeddings_5', 'clean_publicremarks_nopunct_embeddings_6', 'clean_publicremarks_nopunct_embeddings_7', 'clean_publicremarks_nopunct_embeddings_8', 'clean_publicremarks_nopunct_embeddings_9', 'clean_publicremarks_nopunct_embeddings_10', 'clean_publicremarks_nopunct_embeddings_11', 'clean_publicremarks_nopunct_embeddings_12', 'clean_publicremarks_nopunct_embeddings_13', 'clean_publicremarks_nopunct_embeddings_14', 'clean_publicremarks_nopunct_embeddings_15', 'clean_publicremarks_nopunct_embeddings_16', 'clean_publicremarks_nopunct_embeddings_17', 'clean_publicremarks_nopunct_embeddings_18', 'clean_publicremarks_nopunct_embeddings_19', 'clean_publicremarks_nopunct_embeddings_20', 'clean_publicremarks_nopunct_embeddings_21', 'clean_publicremark

['clean_publicremarks_stemmed_embeddings_0', 'clean_publicremarks_stemmed_embeddings_1', 'clean_publicremarks_stemmed_embeddings_2', 'clean_publicremarks_stemmed_embeddings_3', 'clean_publicremarks_stemmed_embeddings_4', 'clean_publicremarks_stemmed_embeddings_5', 'clean_publicremarks_stemmed_embeddings_6', 'clean_publicremarks_stemmed_embeddings_7', 'clean_publicremarks_stemmed_embeddings_8', 'clean_publicremarks_stemmed_embeddings_9', 'clean_publicremarks_stemmed_embeddings_10', 'clean_publicremarks_stemmed_embeddings_11', 'clean_publicremarks_stemmed_embeddings_12', 'clean_publicremarks_stemmed_embeddings_13', 'clean_publicremarks_stemmed_embeddings_14', 'clean_publicremarks_stemmed_embeddings_15', 'clean_publicremarks_stemmed_embeddings_16', 'clean_publicremarks_stemmed_embeddings_17', 'clean_publicremarks_stemmed_embeddings_18', 'clean_publicremarks_stemmed_embeddings_19', 'clean_publicremarks_stemmed_embeddings_20', 'clean_publicremarks_stemmed_embeddings_21', 'clean_publicremark

['clean_publicremarks_lemmatized_embeddings_0', 'clean_publicremarks_lemmatized_embeddings_1', 'clean_publicremarks_lemmatized_embeddings_2', 'clean_publicremarks_lemmatized_embeddings_3', 'clean_publicremarks_lemmatized_embeddings_4', 'clean_publicremarks_lemmatized_embeddings_5', 'clean_publicremarks_lemmatized_embeddings_6', 'clean_publicremarks_lemmatized_embeddings_7', 'clean_publicremarks_lemmatized_embeddings_8', 'clean_publicremarks_lemmatized_embeddings_9', 'clean_publicremarks_lemmatized_embeddings_10', 'clean_publicremarks_lemmatized_embeddings_11', 'clean_publicremarks_lemmatized_embeddings_12', 'clean_publicremarks_lemmatized_embeddings_13', 'clean_publicremarks_lemmatized_embeddings_14', 'clean_publicremarks_lemmatized_embeddings_15', 'clean_publicremarks_lemmatized_embeddings_16', 'clean_publicremarks_lemmatized_embeddings_17', 'clean_publicremarks_lemmatized_embeddings_18', 'clean_publicremarks_lemmatized_embeddings_19', 'clean_publicremarks_lemmatized_embeddings_20', '

In [26]:
df_sales_use2.columns

Index(['index', 'Transprice', 'bedrooms', 'cj_living_area', 'basement',
       'has_pool', 'parking', 'story', 'avmValue', 'avmerror',
       ...
       'clean_publicremarks_lemmatized_embeddings_502',
       'clean_publicremarks_lemmatized_embeddings_503',
       'clean_publicremarks_lemmatized_embeddings_504',
       'clean_publicremarks_lemmatized_embeddings_505',
       'clean_publicremarks_lemmatized_embeddings_506',
       'clean_publicremarks_lemmatized_embeddings_507',
       'clean_publicremarks_lemmatized_embeddings_508',
       'clean_publicremarks_lemmatized_embeddings_509',
       'clean_publicremarks_lemmatized_embeddings_510',
       'clean_publicremarks_lemmatized_embeddings_511'],
      dtype='object', length=3181)

In [27]:
files_to_merge =['noun_sentences/agent_comments_sales_GA_noun_sentence_average_error_clean_publicremarks.fea',
                 'noun_sentences/agent_comments_sales_GA_average_embedding_clean_publicremarks_nounsentences.fea',
                 'noun_sentences/agent_comments_sales_GA_average_embedding_amplitute_clean_publicremarks_nounsentences.fea',
                 'noun_sentences/agent_comments_sales_GA_average_embedding_scaled_clean_publicremarks_nounsentences.fea',
                 'ngrams/agent_comments_sales_GA_ngram_average_error_clean_publicremarks_nopunct_bigram.fea',
                 'ngrams/agent_comments_sales_GA_ngram_average_error_clean_publicremarks_nopunct_fourgram.fea',
                 'ngrams/agent_comments_sales_GA_ngram_average_error_clean_publicremarks_nopunct_trigram.fea',
                 'ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_unigrams.fea',
                 'ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_bigrams.fea',
                 'ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_trigrams.fea',
                 'ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_fourgrams.fea']

In [28]:
for file in files_to_merge:
    print(file)
    df_temp = pd.read_feather(file)
    df_sales_use2 = df_sales_use2.merge(df_temp, on='index')

noun_sentences/agent_comments_sales_GA_noun_sentence_average_error_clean_publicremarks.fea
noun_sentences/agent_comments_sales_GA_average_embedding_clean_publicremarks_nounsentences.fea
noun_sentences/agent_comments_sales_GA_average_embedding_amplitute_clean_publicremarks_nounsentences.fea
noun_sentences/agent_comments_sales_GA_average_embedding_scaled_clean_publicremarks_nounsentences.fea
ngrams/agent_comments_sales_GA_ngram_average_error_clean_publicremarks_nopunct_bigram.fea
ngrams/agent_comments_sales_GA_ngram_average_error_clean_publicremarks_nopunct_fourgram.fea
ngrams/agent_comments_sales_GA_ngram_average_error_clean_publicremarks_nopunct_trigram.fea
ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_unigrams.fea
ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_bigrams.fea
ngrams/agent_comments_sales_GA_ngrams_probabilities_clean_publicremarks_nopunct_trigrams.fea
ngrams/agent_comments_sales_GA_ngrams_probabilities_clea

In [31]:
df_sales_use2

,index,Transprice,bedrooms,cj_living_area,basement,has_pool,parking,story,avmValue,avmerror,...,prob04_column_463,prob04_column_464,prob04_column_465,prob04_column_466,prob04_column_467,prob04_column_468,prob04_column_469,prob04_column_470,prob04_column_471,prob04_column_472
0,128,359000.0,4.0,3295.0,1,0,2,1.5,405972.57,-0.130843,...,1.130816e-06,1.570578e-06,1.696225e-06,1.916106e-06,2.104575e-06,3.046922e-06,3.455273e-06,5.748317e-06,7.821481e-06,1.448073e-05
1,190,265000.0,4.0,2927.0,2,0,0,1.0,244827.67,0.076122,...,9.737586e-07,1.225051e-06,1.350697e-06,1.790459e-06,1.916106e-06,2.104575e-06,7.570188e-06,1.225051e-05,1.671095e-05,2.889864e-05
2,232,239900.0,4.0,2230.0,0,0,2,1.0,229674.21,0.042625,...,1.005170e-06,1.036582e-06,1.225051e-06,1.884694e-06,4.052092e-06,7.538776e-06,9.517705e-06,1.781036e-05,2.264774e-05,5.405617e-04
3,308,130900.0,3.0,1333.0,0,0,2,1.0,143607.40,-0.097077,...,3.141157e-08,3.141157e-08,3.141157e-08,3.141157e-08,6.282314e-08,1.570578e-07,1.036582e-06,8.889474e-06,1.382109e-05,5.635235e-05
4,467,406000.0,4.0,3246.0,0,0,2,2.0,408971.22,-0.007318,...,8.481123e-07,1.036582e-06,1.099405e-06,1.099405e-06,1.759048e-06,1.978929e-06,2.827041e-06,3.109745e-06,6.250902e-06,7.633011e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636144,6908610,163900.0,NaN,1616.0,0,0,2,1.0,174640.18,-0.065529,...,4.083504e-07,4.711735e-07,6.596429e-07,1.005170e-06,1.225051e-06,3.015511e-06,3.989269e-06,6.627841e-06,1.476344e-05,5.597541e-05
636145,6912211,292300.0,3.0,2458.0,0,0,2,1.5,212083.67,0.274432,...,3.141157e-08,3.141157e-08,3.141157e-08,6.282314e-08,6.282314e-08,6.282314e-08,9.423471e-08,9.423471e-08,9.423471e-08,5.025851e-07
636146,6912289,181000.0,3.0,1703.0,0,0,2,1.0,157484.07,0.129922,...,3.141157e-07,4.083504e-07,4.083504e-07,1.005170e-06,1.067993e-06,1.225051e-06,1.476344e-06,2.104575e-06,2.575749e-06,4.963028e-05
636147,6912290,179800.0,3.0,2057.0,0,0,0,1.0,193862.59,-0.078212,...,3.141157e-08,3.141157e-08,3.141157e-08,3.141157e-08,3.141157e-08,3.141157e-08,3.141157e-08,3.141157e-08,6.282314e-08,1.253322e-05


In [30]:
print(len(df_sales_use2.columns))

7909


In [35]:
df_sales_use2.to_feather(file + '_merged.fea') 

In [34]:
print(file + '_merged.fea')

agent_comments_sales_GA_merged.fea


In [1]:
df_sales_use2

NameError: name 'df_sales_use2' is not defined